In [1]:
import torch
import nvtabular as nvt
from nvtabular import dataset as ds
from merlin_models import tf as mtf

import logging
logging.basicConfig(level=logging.INFO)

In [2]:
dataset = ds.MovieLens("/romeyn/data")

# stats = dataset.statistics(transformed=False, split_names="splits")
# stats.display_overview()

In [3]:
data = dataset.transform(for_training=True)
transformed_stats = dataset.statistics(transformed=True)

transformed_stats.display_overview()

INFO:nvtabular:Loading Transforming dataset from cache...
INFO:nvtabular:Loading stats from /romeyn/data/movielens/transformed/stats.pb


In [4]:
# dataset.schema(transformed=True)

In [5]:
dataloader_kwargs = dict(batch_size=20000, separate_labels=True, targets=["rating_binary"])

train_data = mtf.DataLoader.from_directory(data.train, **dataloader_kwargs)
eval_data = mtf.DataLoader.from_directory(data.eval, **dataloader_kwargs)

train_data.output_shapes

{'userId': TensorShape([20000, 1]),
 'genres': TensorShape([20000, 10]),
 'movieId': TensorShape([20000, 1]),
 'rating': TensorShape([20000, 1]),
 'rating_binary': TensorShape([20000, 1])}

In [6]:
inputs = mtf.TabularFeatures.from_column_group(train_data.columns, aggregation="concat")
block = inputs >> mtf.MLPBlock([512, 256])
head = mtf.Head.from_column_group(train_data.columns)
model = block.to_model(head, optimizer="adam")

model

SequentialBlockWithHead(
  (block): SequentialBlock(
    (layers): List(
      (0): TabularFeatures(
        (to_merge): List(
          (0): EmbeddingFeatures(
            (convert_to_sparse): AsSparseFeatures()
            (embeddings): Dict(
              (userId): TableConfig(vocabulary_size=162541, dim=512, initializer=None, optimizer=None, combiner='mean', name='userId')
              (genres): TableConfig(vocabulary_size=20, dim=16, initializer=None, optimizer=None, combiner='mean', name='genres')
              (movieId): TableConfig(vocabulary_size=56585, dim=512, initializer=None, optimizer=None, combiner='mean', name='movieId')
            )
          )
        )
        (aggregation): ConcatFeatures()
      )
      (1): Dense(512, activation=relu, use_bias=True)
      (2): Dense(256, activation=relu, use_bias=True)
    )
  )
  (head): Head(
    (tasks): Dict(
      (rating_binary): PredictionTask(
        (pre): Dense(1, activation=sigmoid, use_bias=True)
        (eval_metri

In [7]:
model.fit(train_data, epochs=3)

Epoch 1/3
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


1001/1001 [==============================] - 50s 44ms/step - rating/rmse: 0.4142 - rating_binary/precision: 0.7613 - rating_binary/recall: 0.8660 - rating_binary/accuracy: 0.7465 - rating_binary/auc: 0.8053 - loss: 0.6822 - regularization_loss: 0.0000e+00 - total_loss: 0.6822
Epoch 2/3
1001/1001 [==============================] - 47s 44ms/step - rating/rmse: 0.3911 - rating_binary/precision: 0.7932 - rating_binary/recall: 0.8671 - rating_binary/accuracy: 0.7756 - rating_binary/auc: 0.8442 - loss: 0.6170 - regularization_loss: 0.0000e+00 - total_loss: 0.6170
Epoch 3/3
1001/1001 [==============================] - 50s 46ms/step - rating/rmse: 0.3732 - rating_binary/precision: 0.8162 - rating_binary/recall: 0.8737 - rating_binary/accuracy: 0.7980 - rating_binary/auc: 0.8716 - loss: 0.5652 - regularization_loss: 0.0000e+00 - total_loss: 0.5652
